# Capstone Project - A Reasonable Location for a New Bookstore in Greater Melbourne

The codes for the project are divided into several parts to fulfill different functions and support the project.

## Part 2, Data Collection - Bookstores

The codes in Part 2 were to:
1. Use Foursquare API to search each LGA using a keyword of "book"
2. Process all bookstore data to remove non-bookstore and duplicated venues
3. Save the dataframe in a csv file for future study

#### Import modules

In [1]:
# !conda install -c conda-forge geopy --yes 
# !conda install -c conda-forge folium=0.5.0 --yes

import requests 
import pandas as pd 
import numpy as np 
import random 
from geopy.geocoders import Nominatim 
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas import json_normalize
import folium 


#### Load dataframe of city Melbourne

In [2]:
df_city=pd.read_csv('city_melbourne.csv')
df_city.head()

latitude=df_city.Latitude[0]
longitude=df_city.Longitude[0]


#### Define Foursquare Credentials and Version based on the query hyperlink

> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**

In [3]:
CLIENT_ID = 'KOLKPBHIWJ2OJXVNUSJUASYIB0B1VAD0Z5JQ5PQGXXK4OHBH' # your Foursquare ID
CLIENT_SECRET = 'F5C4XRNSEN4VQ4J2SMAAXXOMNCFXB023JOBAD4W4PFB1IRHM' # your Foursquare Secret

####
VERSION = '20180604'
search_query = 'book'
LIMIT = 100 # extract max 100 venus
radius = 500000 #m, a very larg value to cover each LGA


#### Search venues for each LGA

In [8]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# count initilization
i=0

for lat,lng,CouncilSeat in zip(df_city['Latitude'],df_city['Longitude'],df_city['CouncilSeat']):
    i=i+1
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, search_query, radius, LIMIT)
    results = requests.get(url).json()
    venues = results['response']['venues']
    #print(url,results,venues)
    
    # tranform venues into a dataframe
    dataframe = json_normalize(venues)
                   
    # keep only columns that include venue name, and anything that is associated with location
    filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
    dataframe_filtered = dataframe.loc[:, filtered_columns]

    # filter the category for each row
    dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

    # clean column names by keeping only last term
    dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

    dataframe_temp1=dataframe_filtered.loc[:,['name','categories','address','lat','lng','distance','city']]

    dataframe_temp2=dataframe_temp1[dataframe_temp1['categories'].isin(["Bookstore","College Bookstore"])]
    dataframe_temp2=dataframe_temp2.reset_index()
    
    if i<=1:
        #print('new')
        df_bookstore=dataframe_temp2.copy(deep=True)
    else:
        #print('append')
        df_bookstore=df_bookstore.append(dataframe_temp2)
    
    print('ID: ',i,' ',CouncilSeat,'| local: ',dataframe_temp2.shape,'| total: ',df_bookstore.shape)

ID:  1   Melbourne | local:  (38, 8) | total:  (38, 8)
ID:  2   St Kilda | local:  (37, 8) | total:  (75, 8)
ID:  3   Malvern | local:  (39, 8) | total:  (114, 8)
ID:  4   Richmond | local:  (35, 8) | total:  (149, 8)
ID:  5   Greensborough | local:  (37, 8) | total:  (186, 8)
ID:  6   Sandringham | local:  (38, 8) | total:  (224, 8)
ID:  7   Camberwell | local:  (39, 8) | total:  (263, 8)
ID:  8   Preston | local:  (37, 8) | total:  (300, 8)
ID:  9   Caulfield North | local:  (40, 8) | total:  (340, 8)
ID:  10   Altona | local:  (38, 8) | total:  (378, 8)
ID:  11   Cheltenham | local:  (36, 8) | total:  (414, 8)
ID:  12   Doncaster | local:  (39, 8) | total:  (453, 8)
ID:  13   Footscray | local:  (37, 8) | total:  (490, 8)
ID:  14   Glen Waverley | local:  (36, 8) | total:  (526, 8)
ID:  15   Moonee Ponds | local:  (35, 8) | total:  (561, 8)
ID:  16   Coburg | local:  (38, 8) | total:  (599, 8)
ID:  17   Nunawading | local:  (36, 8) | total:  (635, 8)
ID:  18   Sunshine | local:  (37

#### Drop duplicated venues

In [5]:
df_bookstore.drop(labels='index', axis=1, inplace=True)
df_bookstore=df_bookstore.drop_duplicates()


#### Save the dataframe into a csv file for future study

In [6]:
df_bookstore.to_csv("city_melbourne_bookstore.csv",index=False)
print(df_bookstore.info())
print(df_bookstore.describe())
df_bookstore.head()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1072 entries, 0 to 32
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        1072 non-null   object 
 1   categories  1072 non-null   object 
 2   address     750 non-null    object 
 3   lat         1072 non-null   float64
 4   lng         1072 non-null   float64
 5   distance    1072 non-null   int64  
 6   city        843 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 67.0+ KB
None
               lat          lng      distance
count  1072.000000  1072.000000   1072.000000
mean    -37.830686   145.011931  14839.641791
std       0.080105     0.113490  12586.297103
min     -38.339334   144.321030     11.000000
25%     -37.848824   144.962280   6218.000000
50%     -37.816048   144.971291  11494.000000
75%     -37.798900   145.054382  18944.000000
max     -37.487703   145.353860  64349.000000


,name,categories,address,lat,lng,distance,city
0,Federation Square Book Market,Bookstore,Federation Square,-37.817229,144.969340,638,Melbourne
1,The Book Grocer,Bookstore,455 Bourke Street,-37.815270,144.960540,258,Melbourne
2,The Book Grocer,Bookstore,287 Little Collins St,-37.814927,144.965139,191,Melbourne
3,The Book Grocer,Bookstore,165 Swanston Street,-37.813913,144.965694,225,Melbourne
4,Book Grocer,Bookstore,206 Bourke St,-37.812910,144.967020,369,Melbourne


#### Plot all bookstores on the map

In [7]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=9) # generate map centred around the Conrad Hotel

for lat, lng in zip(df_bookstore.lat, df_bookstore.lng):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        #popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map